# Proyek Analisis Data: [E-Commerce Public Dataset]
- **Nama:** [Rismano Baharuddin Muhammad]
- **Email:** [rismanobahar@gmail.com]
- **ID Dicoding:** [rismanobahar]

## Menentukan Pertanyaan Bisnis

- Kapan terakhir kali pelanggan melakukan transaksi?
- Seberapa sering seorang pelanggan melakukan pembelian dalam beberapa bulan terakhir?
- Berapa banyak uang yang dihabiskan pelanggan dalam beberapa bulan terakhir?

## Import Semua Packages/Library yang Digunakan

In [1532]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data Wrangling

### Gathering Data

In [ ]:
customers_df = pd.read_csv('data/e-commerce-public-dataset/customers_dataset.csv')
customers_df.head()

In [ ]:
sales_df = pd.read_csv('data/e-commerce-public-dataset/order_items_dataset.csv')
sales_df.head()

In [ ]:
orders_df = pd.read_csv('data/e-commerce-public-dataset/orders_dataset.csv')
orders_df.head()

**Insight:**
- xxx
- xxx

### Assessing Data

In [ ]:
customers_df.info()
customers_df.isna().sum()
print("Jumlah duplikasi: ", customers_df.duplicated().sum())
customers_df.describe()

In [ ]:
sales_df.info()
sales_df.isna().sum()
print("Jumlah duplikasi: ", sales_df.duplicated().sum())
sales_df.describe()

In [ ]:
orders_df.info()
orders_df.isna().sum()
print("Jumlah duplikasi: ", orders_df.duplicated().sum())
orders_df.describe()

**Insight:**
- customers_df = tidak ditemukan masalah
- sales_df = tidak ditemukan masalah
- order_df = terdapat missing values sebesar 160 pada order_approved_at, 1783 pada order_delivered_carrier_date, 2965 pada order_delivered_customer_date

### Cleaning Data

In [ ]:
orders_df[orders_df.order_approved_at.isna()]
orders_df.order_approved_at.value_counts()
orders_df['order_approved_at'].fillna(value="2018-02-27 04:31:10", inplace=True)
orders_df.isna().sum()

In [ ]:
orders_df[orders_df.order_delivered_carrier_date.isna()]
orders_df.order_delivered_carrier_date.value_counts()
orders_df['order_delivered_carrier_date'].fillna(value="2018-05-09 15:48:00", inplace=True)
orders_df.isna().sum()

In [ ]:
orders_df[orders_df.order_delivered_customer_date.isna()]
orders_df.order_delivered_customer_date.value_counts()
orders_df.fillna({'order_delivered_customer_date' : "2018-05-08 19:36:48"}, inplace=True)
orders_df.isna().sum()

**Insight:**
- Missing value berhasil ditangani dengan imputation

## Exploratory Data Analysis (EDA)

### Explore ...

In [ ]:
customers_df.describe(include="all")
customers_df.groupby(by="customer_city").customer_id.nunique().sort_values(ascending=False).head(10)
# customers_df.groupby(by="customer_state").customer_id.nunique().sort_values(ascending=False)

In [ ]:
sales_df.describe(include="all")
sales_df.groupby(by="price").product_id.nunique().sort_values(ascending=False)

In [ ]:
orders_df.describe(include="all")
orders_df["order_purchase_timestamp"] = pd.to_datetime(orders_df["order_purchase_timestamp"]).dt.strftime('%Y-%m-%d')
orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df.order_purchase_timestamp   , format='%Y-%m-%d')
orders_df.groupby(by="order_purchase_timestamp").order_id.nunique().sort_values(ascending=False)

In [ ]:
sales_orders_df = pd.merge(
    left = sales_df,
    right = orders_df,
    how="left",
    left_on="order_id",
    right_on="order_id"
)
sales_orders_df.head()

In [ ]:
all_df = pd.merge(
    left=sales_orders_df,
    right=customers_df,
    how="left",
    left_on="customer_id",
    right_on="customer_id"
)
all_df.head()

**Insight:**
- persebaran jumlah customer berdasarkan kota(city) dan negara bagian(state) terlihat bahwa kota sao paulo mendominasi sebagai customer terbanyak, sedangkan SP adalah negara bagian dengan jumlah customer terbanyak
- diketahui bahwa rata-rata harga barang yang dibeli adalah 120, harga minimum yang dibeli yaitu 0.85, harga tertinggi yang dibeli 6735
- terlihat jumlah pembelian terbanyak adalah pada tanggal 2017-11-24

## Visualization & Explanatory Analysis

### Pertanyaan 1: Berapa banyak uang yang dihabiskan pelanggan dalam beberapa bulan terakhir?


### Pertanyaan 2: Seberapa sering seorang pelanggan melakukan pembelian dalam beberapa bulan terakhir?

### Pertanyaan 3: Kapan terakhir kali pelanggan melakukan transaksi?

In [ ]:
rfm_df = all_df.groupby(by="customer_id", as_index=False).agg({
    "order_purchase_timestamp" : "max",
    "order_id" : "nunique",
    "price" : "sum"
})
rfm_df.columns = ["customer_id", "max_order_timestamp", "frequency", "monetary"]

rfm_df["max_order_timestamp"] = rfm_df["max_order_timestamp"].dt.date
recent_date = orders_df["order_purchase_timestamp"].dt.date.max()
rfm_df["recency"] = rfm_df["max_order_timestamp"].apply(lambda x: (recent_date - x).days)

rfm_df.drop("max_order_timestamp", axis=1, inplace=True)
rfm_df.head()

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(30,6))

colors = ["#72BCD4", "#72BCD4", "#72BCD4", "#72BCD4", "#72BCD4"]

sns.barplot(y="recency", x="customer_id", data=rfm_df.sort_values(by="recency", ascending=True).head(5), palette=colors, ax=ax[0])
ax[0].set_ylabel(None)
ax[0].set_xlabel(None)
ax[0].set_title("By Recency (days)", loc="center", fontsize=18)
ax[0].tick_params(axis="x", labelsize=15)

sns.barplot(y="frequency", x="customer_id", data=rfm_df.sort_values(by="frequency", ascending=False).head(5), palette=colors, ax=ax[1])
ax[1].set_ylabel(None)
ax[1].set_xlabel(None)
ax[1].set_title("By Frequency", loc="center", fontsize=18)
ax[1].tick_params(axis='x', labelsize=15)
 
sns.barplot(y="monetary", x="customer_id", data=rfm_df.sort_values(by="monetary", ascending=False).head(5), palette=colors, ax=ax[2])
ax[2].set_ylabel(None)
ax[2].set_xlabel(None)
ax[2].set_title("By Monetary", loc="center", fontsize=18)
ax[2].tick_params(axis='x', labelsize=15)
 
plt.suptitle("Best Customer Based on RFM Parameters (customer_id)", fontsize=20)
plt.show()

In [1548]:
all_df.to_csv("all_data.csv", index=False)

**Insight:**
- dari visualisasi data diatas kita dapat melihat beberapa pelanggan terbaik berdasarkan ketiga parameter tersebut.

## Conclusion

- Conclution pertanyaan 1: 

Berdasarkan analisis data diatas didapatkan bahwa terdapat 4 pelanggan yang melakukan transaksi terakhir dengan jumlah yang sama

Saran:
Melakukan diskon secara berkala sehingga pelanggan selalu merasa percaya untuk menggunakan e-commerce ini

- Conclution pertanyaan 2:

berdasarkan analisis data diatas terdapat total 5 pelanggan yang paling sering melakukan transaksi
Saran:
Melakukan update serta promisi bulanan, khususnya pada tanggal kembar agar pelanggan tetap aktif bertransaksi

- Conclusion pertanyaan 3:

berdasarkan analisis data diatas dapat dilihat bahwa terdapat satu pelanggan yang telah melakukan transaksi senilai 14,000$
saran:
Memberi pelanggan hadiah dimana apabila pelanggan semakin meningkatkan transaksinya, maka akan semakin besar pula hadiah yang pelanggan tersebut peroleh